# Create organic alkalinity with citric acid (citrate)

In [ ]:
# use a database with some organic molecules 

# therefor the database needs to be installed first and then put in the phreeqpython folder


#print(phreeqpython.__path__)

#check this path and put the  'minteq.v4.dat'  file in there 

# https://github.com/Vitens/phreeqpython/issues/2


In [ ]:
#titration plots for different solutions


from phreeqpython import PhreeqPython 

import numpy as np


from matplotlib import pyplot as plt

import pandas as  pd


#produce a solution with NaOH one in equilibrium with atmophere and the other one pure


# use the vitens database for all constants







# geneal phreeqc info

Note your input is  mol/L, so the density may be important. PHREEQC will
use the density to converting to mol/kgw, which are the units used for all
internal calculations (and all printout). Right now the density is 1.0.

In [ ]:
# some unecessary style settings

plt.rcParams.update({'font.size': 22,
                     #font.weight':'bold',
                     'xtick.labelsize' : 18,
                     'ytick.labelsize' : 18,
                     'lines.linewidth' :  4,
                     'axes.labelsize'  : 20.0,
                     'axes.titlesize':  20,
                     'figure.titlesize': 'x-large',
                     'legend.fontsize': 18.0,})



In [ ]:
#produce a solution with NaOH one in equilibrium with atmophere and the other one pure


# use the vitens database for all constants

# create new PhreeqPython instance

pp = PhreeqPython(database='minteq.v4.dat')

#for differnt NaOH doses

temp=25

#concetration 50mmol/l 0.05mol/l
c=10

c_org_acid=3

# Line 5: units concentration units

#units--Indicates default concentration units are entered on this line. Optionally, -u[ nits].

#concentration units --Default concentration units. Three groups of concentration units are allowed, concentration (1) per liter ("/L"), (2) per kilogram solution ("/kgs"), or (3) per kilogram water ("/kgw"). All concentration units for a solution must be within the same group. Within a group, either grams or moles may be used, and prefixes milli (m) and micro (u) are acceptable. Parts per thousand, "ppt"; parts per million, "ppm"; and parts per billion, "ppb", are acceptable in the "per kilogram solution" group. Default is mmol/kgw (millimoles per kilogram water).


# set the Sodium moles and produce charge balance for oH-

solution1=pp.add_solution_simple({'NaOH':c},temperature=temp, units='mmol')



#concentration CO2
p=415 # ppm

#partial pressure pCO2 in atm
pCO2=p*1e-6 


#phreeqc always uses log10 values
input_pCO2=np.log10(pCO2)
    
    
# equalize solution with CO2
# equilibrium with atmosphere
solution1.equalize(['CO2(g)'], [input_pCO2])


#take just a part of this solution



solution2=pp.add_solution_simple({'NaOH':c},temperature=temp, units='mmol')


#create a third solution with some organic acids inside
# but under atmospheric CO2 pressure

solution3=pp.add_solution_simple({'NaOH':c+c_org_acid},temperature=temp, units='mmol')

solution3.add('H3(Citrate)', c_org_acid, 'mmol')  #add the citic acid    

solution3.equalize(['CO2(g)'], [input_pCO2])


#produce acetic acid organic alkalinity
solution4=pp.add_solution_simple({'NaOH':c+c_org_acid},temperature=temp, units='mmol')

solution4.add('H(Acetate)',c_org_acid , 'mmol')  #add the  acetic acid 

solution4.equalize(['CO2(g)'], [input_pCO2])


#produce formic acid (the simplest and srongest organic acid)



solution5=pp.add_solution_simple({'NaOH':c+c_org_acid},temperature=temp, units='mmol')

solution5.add('H(Formate)',c_org_acid , 'mmol')  #add the  acetic acid 

solution5.equalize(['CO2(g)'], [input_pCO2])

#acid used for titration
# 1 liter of this solution
solution_acid=pp.add_solution_simple({'HCl':20},temperature=temp, units='mmol')



In [ ]:
solution1

In [ ]:
solution2

In [ ]:
# Calculate the total volume of the mixed solution
total_volume = 100.0  # ml

# Calculate the volumes of each solution to mix
nacl_volume = total_volume * 0.6
naoh_volume = total_volume * 0.4

# Create a mixture of the two solutions

#volumetric or weight based mix
mix= solution1 * 0.5 + solution_acid * 0.5



# Try to add acid volume instead of acid moles

How ?

## Resize/Rescale every input
Because phreeqc is always calculating reactions with 1 Liter I need to resize all my imputs to model the concentrations and amounts per 1 Liter

In [ ]:
# add acid stepwise 

# acid steps
dx=0.1 #ml


pH_s1=[]

pH_s2=[]

pH_s3=[]

pH_s4=[]

pH_s5=[]

acid_s=[]




for acid in range(300):
    
    pH_s1.append(solution1.pH)
    
    pH_s2.append(solution2.pH)
    
    pH_s3.append(solution3.pH)
    
    pH_s4.append(solution4.pH)
    
    pH_s5.append(solution5.pH)

    acid_s.append(acid*0.1)

    
    #solution1=solution1+dx*solution_acid
    
    solution1.add('HCl',0.1, 'mmol') # add 1 mmol of 
    
    solution2.add('HCl',0.1, 'mmol') # add 1 mmol of 
    
    solution3.add('HCl',0.1, 'mmol') # add 1 mmol of 
    
    solution4.add('HCl',0.1, 'mmol') # add 1 mmol of 
    
    solution5.add('HCl',0.1, 'mmol') # add 1 mmol of 
    
    

In [ ]:
fig = plt.figure(figsize=[16,9])

plt.plot(acid_s,pH_s2, label='pure NaOH={:} mmol/kg solution'.format(c))

plt.plot(acid_s,pH_s1, label=' same NaOH={:} mmol/kg solution in atm equilibrium'.format(c))

plt.plot(acid_s,pH_s3, label='NaOH={:} mmol/l solution+ \n citric acid H3(Citrate)={:} mmol/kg + atm equilibrium'.format(c+c_org_acid,c_org_acid)) 

plt.plot(acid_s,pH_s4, label='NaOH={:} mmol/l solution+ \n acetic acid H(Acetate)={:} mmol/kg + atm equilibrium'.format(c+c_org_acid,c_org_acid)) 

plt.plot(acid_s,pH_s5, label='NaOH={:} mmol/l solution+ \n formic acid H(Formate)={:} mmol/kg + atm equilibrium'.format(c+c_org_acid,c_org_acid))


plt.ylabel('pH')

plt.xlabel('added acid [mmol]')

plt.legend(loc='upper right')

plt.title('Different solutions organic inorganic')


plt.savefig('plots/organic_anions_titration-plot.pdf')  
plt.savefig('plots/organic_anions_titration-plot.svg')  

# Example of mixing two solutions

In [ ]:
solution1 = pp.add_solution_simple({'NaCl':1})
solution2 = pp.add_solution_simple({'NaCl':3})
# make a solution of 50% solution 1 and 50% solution 2:
solution3 = solution1 * 0.5 + solution2 * 0.5
# make a solution by mixing solution 1 and 2 together
solution4 = solution1 + solution2